<a href="https://colab.research.google.com/github/guilhermecarva/guilhermecarva/blob/main/Tree_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso.

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [ ]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10.

Como podemos proceder? O código abaixo exemplifica um jeito:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.33071301671575365  /  0.016331523412902806
estimators:  10  prof:  5  | R2 mean / std:  0.40509514070321223  /  0.018060549434403612
estimators:  10  prof:  10  | R2 mean / std:  0.3505103448869891  /  0.04517698379594533
estimators:  100  prof:  1  | R2 mean / std:  0.3383163077435632  /  0.03427873765177229
estimators:  100  prof:  5  | R2 mean / std:  0.4225551623014924  /  0.04643089369666596
estimators:  100  prof:  10  | R2 mean / std:  0.4161326445040172  /  0.033813371472341675
estimators:  100  prof:  1  | R2 mean / std:  0.32819176781854936  /  0.04271920094793037
estimators:  100  prof:  5  | R2 mean / std:  0.4230130416784485  /  0.03459543157520171
estimators:  100  prof:  10  | R2 mean / std:  0.43114628801978155  /  0.04012120590813385


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso.
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [ ]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [ ]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [ ]:
grid = GridSearchCV(estimator = estimador_base,
                    param_grid = params_RF,
                    scoring = 'r2',
                    cv = 3)

grid

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes.

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [ ]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [ ]:
grid.best_score_

0.42444658502741595

In [ ]:
grid.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
grid.cv_results_

{'mean_fit_time': array([0.01412272, 0.90284522, 0.01294303, 1.22760916]),
 'std_fit_time': array([2.59640423e-04, 7.94946362e-03, 6.69247364e-05, 1.62409492e-02]),
 'mean_score_time': array([0.00116746, 0.03209265, 0.00080196, 0.03886215]),
 'std_score_time': array([1.23910753e-04, 5.61650156e-04, 8.15362106e-06, 1.52093834e-03]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.30532442, 0.3511556 , 0.29536921, 0.37863526]),
 'split1_test_score': array([0.46541789, 0.44720904, 0.38796397, 0.4491157 ]),
 'split2_tes

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [ ]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
enc = OneHotEncoder()

Vamos inicar o tratamento dos dados

In [ ]:
df['proximidade_ao_mar'].value_counts()

<1H OCEANO       9136
INTERIOR         6551
PERTO OCEANO     2658
PERTO DA BAÍA    2290
ILHA                5
Name: proximidade_ao_mar, dtype: int64

In [ ]:
df.isnull().sum()

longitude                    0
latitude                     0
idade_mediana_das_casas      0
total_comodos                0
total_quartos              207
populacao                    0
familias                     0
salario_mediano              0
preco_mediano_das_casas      0
proximidade_ao_mar           0
dtype: int64

In [ ]:
df = df.dropna(axis = 0)
print(df.shape)

(20433, 10)


In [ ]:
df['proximidade_ao_mar'] = enc.fit_transform(df[['proximidade_ao_mar']]).toarray()

In [ ]:
df.head()

,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0.0


In [ ]:
df['proximidade_ao_mar'].value_counts()

0.0    11399
1.0     9034
Name: proximidade_ao_mar, dtype: int64

Fazendo o split entre treino e teste para utilização em nossos modelos

In [ ]:
X = df.drop(['preco_mediano_das_casas'], axis=1)
y = df['preco_mediano_das_casas']

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(15324, 9) (5109, 9) (15324,) (5109,)


Fazendo o preprocessamento  / normalização dos dados

In [ ]:
sc = StandardScaler()

In [ ]:
sc.fit_transform(Xtrain)

array([[ 0.28386351, -0.11020303, -1.40115964, ...,  3.7549558 ,
        -1.13798902, -0.8943484 ],
       [ 1.3076607 , -1.3710388 , -0.84488593, ...,  0.14184181,
        -0.74541525,  1.11813249],
       [ 0.62845866, -1.071063  , -0.12967689, ..., -0.43921454,
        -0.90269742, -0.8943484 ],
       ...,
       [ 0.57851734, -0.76171296,  1.06233819, ..., -0.35733841,
        -0.40732175,  1.11813249],
       [-1.22935867,  0.9069024 , -1.32169196, ...,  1.4782714 ,
         0.74487015, -0.8943484 ],
       [-1.42412984,  0.98189635,  1.85701491, ...,  0.39539367,
         0.01083406, -0.8943484 ]])

In [ ]:
sc.transform(Xtest)

array([[ 1.16782498, -1.33354183, -0.68595059, ..., -0.64258426,
        -0.36268478, -0.8943484 ],
       [-0.85479872,  1.11313576, -2.11636868, ...,  1.21415488,
         0.32334551, -0.8943484 ],
       [ 0.71835305, -0.80389706,  0.1881938 , ...,  0.80213311,
        -0.34168224,  1.11813249],
       ...,
       [-0.70996888,  0.9490865 ,  1.14180586, ..., -0.1988685 ,
        -0.32089025, -0.8943484 ],
       [-1.10450535,  1.42717293, -0.3680799 , ...,  0.96852652,
        -0.56112982, -0.8943484 ],
       [-0.65503342,  0.89284104,  0.10872613, ..., -1.00970622,
        -0.84479568, -0.8943484 ]])

Instanciando os modelos e definindo os parâmetros para serem utilizados no GridSearch

In [ ]:
#definindo o estimador base
rfr = RandomForestRegressor()
knn = KNeighborsRegressor()
gbr = GradientBoostingRegressor()

#definindo o dicionario de parâmetros do modelo
params_rfr = {"n_estimators":[10,100,1000], "max_depth":[2,5,10]}
params_knn = {"n_neighbors":[3,5,10]}
params_gbr = {"n_estimators":[10,100,1000], "min_samples_split":[2,5], "max_depth":[2,5,10], "random_state":[42]}

In [ ]:
gridrfr = GridSearchCV(estimator = rfr,
                    param_grid = params_rfr,
                    scoring = 'r2',
                    cv = 3,
                    n_jobs=-1)

In [ ]:
gridknn = GridSearchCV(estimator = knn,
                    param_grid = params_knn,
                    scoring = 'r2',
                    cv = 3,
                    n_jobs=-1)

In [ ]:
gridgbr = GridSearchCV(estimator = gbr,
                    param_grid = params_gbr,
                    scoring = 'r2',
                    cv = 3,
                    n_jobs=-1)

In [ ]:
gridrfr, gridknn, gridgbr

(GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
              param_grid={'max_depth': [2, 5, 10],
                          'n_estimators': [10, 100, 1000]},
              scoring='r2'),
 GridSearchCV(cv=3, estimator=KNeighborsRegressor(), n_jobs=-1,
              param_grid={'n_neighbors': [3, 5, 10]}, scoring='r2'),
 GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=-1,
              param_grid={'max_depth': [2, 5, 10], 'min_samples_split': [2, 5],
                          'n_estimators': [10, 100, 1000],
                          'random_state': [42]},
              scoring='r2'))

In [ ]:
#treinando os modelos no grid
gridrfr.fit(Xtrain, ytrain), gridknn.fit(Xtrain, ytrain), gridgbr.fit(Xtrain, ytrain)

(GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
              param_grid={'max_depth': [2, 5, 10],
                          'n_estimators': [10, 100, 1000]},
              scoring='r2'),
 GridSearchCV(cv=3, estimator=KNeighborsRegressor(), n_jobs=-1,
              param_grid={'n_neighbors': [3, 5, 10]}, scoring='r2'),
 GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=-1,
              param_grid={'max_depth': [2, 5, 10], 'min_samples_split': [2, 5],
                          'n_estimators': [10, 100, 1000],
                          'random_state': [42]},
              scoring='r2'))

In [ ]:
gridrfr.best_params_ , gridknn.best_params_, gridgbr.best_params_

({'max_depth': 10, 'n_estimators': 1000},
 {'n_neighbors': 10},
 {'max_depth': 5,
  'min_samples_split': 5,
  'n_estimators': 1000,
  'random_state': 42})

In [ ]:
gridrfr.best_score_ , gridknn.best_score_ , gridgbr.best_score_

(0.7817882100950194, 0.28980330464739007, 0.8248911482483173)

In [ ]:
gridrfr.best_estimator_ , gridknn.best_estimator_ , gridgbr.best_estimator_

(RandomForestRegressor(max_depth=10, n_estimators=1000),
 KNeighborsRegressor(n_neighbors=10),
 GradientBoostingRegressor(max_depth=5, min_samples_split=5, n_estimators=1000,
                           random_state=42))

__Observando os scores ( a métrica utilizada foi o r2 ), podemos concluir que o modelo GradientBooster obteve o maior score e seus melhores parâmetros foram: 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 1000__

__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada.
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [ ]:
class gridSearchAll():

    def __init__(self, grid_models, scoring, cv):
        self.grid_models = []
        self.scoring = scoring
        self.cv = cv
        self.grids = []
        self.score = {}


    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])

    def fit_all(self, X, y):
        for estimator_base, param_grid in self.grid_models:
            grid = GridSearchCV(estimator = estimator_base,
                   param_grid = param_grid,
                   scoring = self.scoring,
                   cv = self.cv,
                   n_jobs=-1)
            grid.fit(X,y)
            self.grids.append(grid)
            self.score[estimator_base] = grid.best_score_
            print("Os melhores parâmetros foram :")
            print(estimator_base)
            print(grid.best_params_)

    def best_all_grid_models(self):
        print("Os scores dos modelos foram :")
        print(self.score)
        print("")
        print("Sendo o melhor modelo : ")
        melhor = max(self.score, key=self.score.get)
        return melhor

In [ ]:
gd = gridSearchAll(2, 'r2', 3)

In [ ]:
gd.grid_models

[]

In [ ]:
params_RF = {'n_estimators': [10, 1000], 'max_depth': [2, 10]}

In [ ]:
gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)

In [ ]:
gd.grid_models

[[RandomForestRegressor(), {'n_estimators': [10, 1000], 'max_depth': [2, 10]}]]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = {"n_neighbors":[1,2,10]})

In [ ]:
gd.grid_models

[[RandomForestRegressor(), {'n_estimators': [10, 1000], 'max_depth': [2, 10]}],
 [KNeighborsRegressor(), {'n_neighbors': [1, 2, 10]}]]

In [ ]:
gd.fit_all(X,y)

Os melhores parâmetros foram :
RandomForestRegressor()
{'max_depth': 10, 'n_estimators': 1000}
Os melhores parâmetros foram :
KNeighborsRegressor()
{'n_neighbors': 10}


In [ ]:
gd.best_all_grid_models()

Os scores dos modelos foram :
{RandomForestRegressor(): 0.568078104356985, KNeighborsRegressor(): 0.2762393249417914}

Sendo o melhor modelo : 


RandomForestRegressor()

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

In [ ]:
gd3 = gridSearchAll(3, 'r2', 3)

In [ ]:
gd3.insert_model(estimator_base = rfr, param_grid = params_rfr)
gd3.insert_model(estimator_base = knn, param_grid = params_knn)
gd3.insert_model(estimator_base = gbr, param_grid = params_gbr)

In [ ]:
gd3.grid_models

[[RandomForestRegressor(),
  {'n_estimators': [10, 100, 1000], 'max_depth': [2, 5, 10]}],
 [KNeighborsRegressor(), {'n_neighbors': [3, 5, 10]}],
 [GradientBoostingRegressor(),
  {'n_estimators': [10, 100, 1000],
   'min_samples_split': [2, 5],
   'max_depth': [2, 5, 10],
   'random_state': [42]}]]

In [ ]:
gd3.fit_all(X,y)

Os melhores parâmetros foram :
RandomForestRegressor()
{'max_depth': 10, 'n_estimators': 1000}
Os melhores parâmetros foram :
KNeighborsRegressor()
{'n_neighbors': 10}
Os melhores parâmetros foram :
GradientBoostingRegressor()
{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}


In [ ]:
gd3.best_all_grid_models()

Os scores dos modelos foram :
{RandomForestRegressor(): 0.5680757898027905, KNeighborsRegressor(): 0.2762393249417914, GradientBoostingRegressor(): 0.6514718800438686}

Sendo o melhor modelo : 


GradientBoostingRegressor()